<h1>Capstone Week-3 Assignment<h1>

<h2>In the below cell I Used Beautiful Soup to parse the URL and extract the table<h2>

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

response_can=requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(response_can.text,"html.parser")

print("HTML extracted........")

HTML extracted........


<h2>Below cell gives the shape of actual Wikipedia data frame<h2>

In [2]:
Table_Rows = soup.find("table", {"class": "wikitable sortable"}).find_all("tr")

Table_Head = []
Postcode= []
Borough = []
Neighbourhood = []

for tab_row in Table_Rows[:1]:
    for i in range(0,3):
        table_head = tab_row.find_all("th")[i].text.strip('\n')
        Table_Head.append(table_head)

for tab_row in Table_Rows[1:]:
    postcode = tab_row.find_all("td")[0].text.strip('\n')
    borough = tab_row.find_all("td")[1].text.strip('\n')
    neighbourhood = tab_row.find_all("td")[2].text.strip('\n')
    
    Postcode.append(postcode)
    Borough.append(borough)
    Neighbourhood.append(neighbourhood)

print("Table Extracted from wikipedia...")
print("   Number of Postcodes:{} \n   Number of Boroughs:{} \n   Number of Neighbourhoods:{}\n".format(len(Postcode),len(Borough),len(Neighbourhood)))

df_Data = pd.DataFrame({Table_Head[0]:Postcode,Table_Head[1]:Borough,Table_Head[2]:Neighbourhood})

print("Shape of Actual Wiki Dataframe:",df_Data.shape)

Table Extracted from wikipedia...
   Number of Postcodes:287 
   Number of Boroughs:287 
   Number of Neighbourhoods:287

Shape of Actual Wiki Dataframe: (287, 3)


<h2>Below cell gives the shape of actual Wikipedia data frame after dropping "Not Assigned" rows<h2>

In [3]:
df_Data_final_noNA = df_Data[(df_Data.Borough !='Not assigned')& (df_Data.Neighbourhood !='Not assigned')].reset_index().drop(['index'],axis=1)
df_Data_final_Ngbhd_NA = df_Data[(df_Data.Borough !='Not assigned') & (df_Data.Neighbourhood =='Not assigned')].reset_index().drop(['index'],axis=1)
df_Data_final_Ngbhd_NA['Neighbourhood']=df_Data_final_Ngbhd_NA['Borough']

df_Data_final =df_Data_final_noNA.append(df_Data_final_Ngbhd_NA)

print("\nShape of Actual Wiki Dataframe after dropping \"not Assigned\" rows:{}\n".format(df_Data_final.shape))


Shape of Actual Wiki Dataframe after dropping "not Assigned" rows:(210, 3)



<h2>Below cell gives the shape of actual Wikipedia data frame after combining Neighbhourhoods<h2>

In [4]:
df_Toronto = pd.DataFrame(columns=['Postcode', 'Borough', 'Neighbourhood'])
Unique_postcode=df_Data_final['Postcode'].unique()

for postcode in Unique_postcode:
    df_postcode_unq=df_Data_final[df_Data_final.Postcode==postcode].reset_index().drop(['index'],axis=1)
    combined_neighoods=df_postcode_unq['Neighbourhood'].str.cat(sep=',', na_rep='?')
    df_Toronto=df_Toronto.append({'Postcode': df_postcode_unq.iloc[0,0], 'Borough': df_postcode_unq.iloc[0,1], 'Neighbourhood': combined_neighoods}, ignore_index=True)
    
print("Shape of Wiki Dataframe after combining Neighbourhoods:{}\n".format(df_Toronto.shape))

Shape of Wiki Dataframe after combining Neighbourhoods:(103, 3)



<h2>Below cell gives the shape of actual Wikipedia data frame after adding Latitude and Longitude Columns<h2>

In [5]:
Df_Lat_Long=pd.read_csv("Geospatial_Coordinates.csv")
Df_Lat_Long.sort_values(by=['Postal Code'],inplace=True)
df_Toronto_lat_long=df_Toronto.sort_values(by=['Postcode']).reset_index().drop(['index'],axis=1)

df_Toronto_lat_long['Latitude'] = Df_Lat_Long['Latitude']
df_Toronto_lat_long['Longitude'] = Df_Lat_Long['Longitude']
print("\nShape of Final Data Frame with latitude and longitude Details:{}\n".format(df_Toronto_lat_long.shape))


Shape of Final Data Frame with latitude and longitude Details:(103, 5)



In [7]:
df_Toronto_lat_long.head(3)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711


<h2>Below Cells explore and cluster the neighborhoods in Toronto. 
Replicated the same analysis as the New York City data.
    
Analysis is limited to Boroughs that contain string - Toronto<h2>

In [8]:
import folium
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
import json
from pandas.io.json import json_normalize

In [9]:
df_Toronto_lat_long['Borough'].unique()
Toronto_data = df_Toronto_lat_long[df_Toronto_lat_long['Borough'].str.contains('Toronto') ].reset_index(drop=True)
Toronto_data.head()  # only boroughs that contain toronto

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [10]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="trt_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Toronto_data['Latitude'], Toronto_data['Longitude'], Toronto_data['Borough'], Toronto_data['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [32]:
CLIENT_ID = '++++++' # your Foursquare ID
CLIENT_SECRET = '______' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ++++++
CLIENT_SECRET:______


In [12]:
Toronto_data.loc[0, 'Neighbourhood']

'The Beaches'

In [13]:
neighborhood_latitude = Toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = Toronto_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


In [14]:
LIMIT = 100
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [15]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e4244c80f5968001b8e6034'},
 'response': {'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.680857404499996,
    'lng': -79.28682091449052},
   'sw': {'lat': 43.67185739549999, 'lng': -79.29924148550948}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd461bc77b29c74a07d9282',
       'name': 'Glen Manor Ravine',
       'location': {'address': 'Glen Manor',
        'crossStreet': 'Queen St.',
        'lat': 43.67682094413784,
        'lng': -79.29394208780985,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.67682094413784,
          'lng': -79.29394208780985}],
        'distanc

In [16]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [17]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Upper Beaches,Neighborhood,43.680563,-79.292869


In [28]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [19]:
Toronto_venues = getNearbyVenues(names=Toronto_data['Neighbourhood'],
                                   latitudes=Toronto_data['Latitude'],
                                   longitudes=Toronto_data['Longitude']
                                  )
print(Toronto_venues.shape)
Toronto_venues.head()

The Beaches
The Danforth West,Riverdale
The Beaches West,India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Roselawn
Forest Hill North,Forest Hill West
The Annex,North Midtown,Yorkville
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
Dovercourt Village,Dufferin
Little Portugal,Trinity
Brockton,Exhibition Place,Parkdale Village
High Park,The Junction South
Parkdale,Roncesvalles
Runnymede

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West,Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [20]:
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Berczy Park,55,55,55,55,55,55
"Brockton,Exhibition Place,Parkdale Village",24,24,24,24,24,24
Business Reply Mail Processing Centre 969 Eastern,17,17,17,17,17,17
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",16,16,16,16,16,16
"Cabbagetown,St. James Town",42,42,42,42,42,42
Central Bay Street,84,84,84,84,84,84
"Chinatown,Grange Park,Kensington Market",80,80,80,80,80,80
Christie,18,18,18,18,18,18


In [21]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 229 uniques categories.


In [22]:
import numpy as np

# one hot encoding
toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood']

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
toronto_onehot.shape

(1706, 229)

In [24]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()
toronto_grouped.shape

(39, 229)

In [25]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 4})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')


----Adelaide,King,Richmond----
              venue  freq
0       Coffee Shop  0.07
1               Bar  0.04
2        Steakhouse  0.04
3              Café  0.04
4  Asian Restaurant  0.03


----Berczy Park----
          venue    freq
0   Coffee Shop  0.0727
1  Cocktail Bar  0.0545
2          Café  0.0364
3    Steakhouse  0.0364
4      Beer Bar  0.0364


----Brockton,Exhibition Place,Parkdale Village----
            venue    freq
0  Breakfast Spot  0.0833
1            Café  0.0833
2     Coffee Shop  0.0833
3       Nightclub  0.0833
4     Yoga Studio  0.0417


----Business Reply Mail Processing Centre 969 Eastern----
                  venue    freq
0      Recording Studio  0.0588
1         Garden Center  0.0588
2  Fast Food Restaurant  0.0588
3            Skate Park  0.0588
4                  Park  0.0588


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
                 venue    freq
0      Airport Service  0.1875
1     Airport

In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [27]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Steakhouse,Bar,Café,Cosmetics Shop,Breakfast Spot,Asian Restaurant,Burger Joint,Restaurant,Thai Restaurant
1,Berczy Park,Coffee Shop,Cocktail Bar,Farmers Market,Seafood Restaurant,Steakhouse,Café,Cheese Shop,Beer Bar,Bakery,Breakfast Spot
2,"Brockton,Exhibition Place,Parkdale Village",Coffee Shop,Café,Breakfast Spot,Nightclub,Yoga Studio,Bakery,Performing Arts Venue,Pet Store,Convenience Store,Climbing Gym
3,Business Reply Mail Processing Centre 969 Eastern,Park,Burrito Place,Butcher,Fast Food Restaurant,Farmers Market,Auto Workshop,Recording Studio,Spa,Restaurant,Pizza Place
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Service,Airport Terminal,Airport Lounge,Sculpture Garden,Rental Car Location,Harbor / Marina,Bar,Boat or Ferry,Coffee Shop,Airport Gate


In [28]:
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

# set number of clusters
kclusters = 20

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]


# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = Toronto_data

toronto_merged.head()


#merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,4,Trail,Health Food Store,Pub,Women's Store,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,11,Greek Restaurant,Coffee Shop,Italian Restaurant,Restaurant,Ice Cream Shop,Furniture / Home Store,Yoga Studio,Pub,Bakery,Dessert Shop
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,17,Park,Sushi Restaurant,Board Shop,Brewery,Italian Restaurant,Liquor Store,Burger Joint,Burrito Place,Ice Cream Shop,Fast Food Restaurant
3,M4M,East Toronto,Studio District,43.659526,-79.340923,1,Café,Coffee Shop,Gastropub,Bakery,Brewery,Italian Restaurant,American Restaurant,Comfort Food Restaurant,Sandwich Place,Cheese Shop
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,5,Bus Line,Lake,Swim School,Park,Colombian Restaurant,Comfort Food Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


In [31]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h1>Cluster-1<h1>

In [33]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
24,Central Toronto,0,Café,Sandwich Place,Coffee Shop,Park,BBQ Joint,History Museum,Middle Eastern Restaurant,Indian Restaurant,Burger Joint,Cosmetics Shop


<h2> Cluster-2<h2>

In [34]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,East Toronto,1,Café,Coffee Shop,Gastropub,Bakery,Brewery,Italian Restaurant,American Restaurant,Comfort Food Restaurant,Sandwich Place,Cheese Shop
11,Downtown Toronto,1,Restaurant,Coffee Shop,Bakery,Café,Pizza Place,Italian Restaurant,Pub,Pharmacy,Playground,Snack Place
13,Downtown Toronto,1,Coffee Shop,Pub,Park,Café,Bakery,Mexican Restaurant,Restaurant,Breakfast Spot,Event Space,Hotel
15,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Cocktail Bar,Cosmetics Shop,American Restaurant,Clothing Store,Hotel,Breakfast Spot,Bakery
16,Downtown Toronto,1,Coffee Shop,Cocktail Bar,Farmers Market,Seafood Restaurant,Steakhouse,Café,Cheese Shop,Beer Bar,Bakery,Breakfast Spot
17,Downtown Toronto,1,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Ice Cream Shop,Juice Bar,Burger Joint,Japanese Restaurant,Bar,Salad Place
19,Downtown Toronto,1,Coffee Shop,Aquarium,Italian Restaurant,Café,Hotel,Restaurant,Fried Chicken Joint,Scenic Lookout,Brewery,Sporting Goods Shop
20,Downtown Toronto,1,Coffee Shop,Café,Hotel,Restaurant,Steakhouse,Seafood Restaurant,American Restaurant,Bar,Italian Restaurant,Deli / Bodega
21,Downtown Toronto,1,Coffee Shop,Café,Hotel,Restaurant,Deli / Bodega,Gym,Bakery,Seafood Restaurant,Steakhouse,Gastropub
28,Downtown Toronto,1,Coffee Shop,Café,Hotel,Restaurant,Beer Bar,Seafood Restaurant,Cocktail Bar,Italian Restaurant,Japanese Restaurant,Cheese Shop


<h2> Cluster-3<h2>

In [35]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Downtown Toronto,2,Park,Playground,Trail,Cupcake Shop,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store


<h2> Cluster-4<h2>

In [36]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,3,Health & Beauty Service,Pool,Garden,Women's Store,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


<h2> Cluster-5<h2>

In [37]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,4,Trail,Health Food Store,Pub,Women's Store,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


<h3>Most common venue seems to be coffe shop<h3>